In [ ]:
pip install dune-client

In [ ]:
from dotenv import load_dotenv

load_dotenv(".env")

In [ ]:
import os
from dune_client.client import DuneClient

DUNE_API_KEY=os.getenv("DUNE_API_KEY")
dune = DuneClient(DUNE_API_KEY)
query_result = dune.get_latest_result(3888983)
print(query_result.result)

In [ ]:
from dune_client.types import QueryParameter
from dune_client.client import DuneClient
from dune_client.query import QueryBase

In [ ]:
query = QueryBase(
    name="ethereum_get_info_from_symbol",
    query_id=3893389,
    params=[
        QueryParameter.text_type(name="symbol", value="usdt"),
    ],
)
results_df = dune.run_query_dataframe(query)
print(results_df.to_json())

In [ ]:
from web3 import Web3

infura_url = f"https://mainnet.infura.io/v3/{os.getenv('INFURA_API_KEY')}"
web3 = Web3(Web3.HTTPProvider(infura_url))

def is_contract_address(address):
    code = web3.eth.get_code(Web3.to_checksum_address(address))
    return code != b''

print(is_contract_address("0xC0ffeEBABE5D496B2DDE509f9fa189C25cF29671"))

In [ ]:
from dune_tools import top_10_eth_transaction

result=top_10_eth_transaction.invoke({"min_amout":1000})
print(result)

In [ ]:
from dune_tools import get_token_balances_of_address
import json

token_balances_str = get_token_balances_of_address.invoke(
    {"address": "0xdfd5293d8e347dfe59e90efd55b2956a1343963d"}
)
token_balances = json.loads(token_balances_str)
    # 生成Token余额分析报表
token_balance_report = []
for token in token_balances:
    balance_usd = token.get("balance_usd", "N/A")
    token_balance_report.append(
        f"Token: {token['token_symbol']} (Contract: {token['token_address']}, Liquidity on DEX in 3 months:${token['total_liquidity_usd']})\n"
        f"Balance: {token['balance']} (USD: {balance_usd} Price:${token['price']})\n"
    )

# Token余额分析部分
token_analysis = "\n".join(token_balance_report)
print(token_analysis)

In [ ]:
import requests
import time
import os

def get_address_tags_from_etherscan(addresses):
    ETHERSCAN_API_KEY = os.getenv("ETHERSCAN_API_KEY")  # Replace with your actual Etherscan API key
    ETHERSCAN_API_URL = 'https://api.etherscan.io/api'
    
    address_tags = {}
    
    for address in addresses:
        params = {
            'module': 'account',
            'action': 'txlist',
            'address': address,
            'apikey': ETHERSCAN_API_KEY,
            'startblock': 0,
            'endblock': 99999999,
            'page': 1,
            'offset': 10,  # Example to limit query, adjust as necessary
            'sort': 'asc'
        }
        
        response = requests.get(ETHERSCAN_API_URL, params=params)
        
        if response.status_code == 200:
            data = response.json()
            if data['status'] == '1':
                # Extracting labels from the result
                address_tags[address] = data['result']
            else:
                address_tags[address] = 'No data found'
        else:
            address_tags[address] = f"Error: {response.status_code}"
        
        # Respect Etherscan API rate limits
        time.sleep(1)
    
    return address_tags

# Example usage
addresses = [
    "0x742d35Cc6634C0532925a3b844Bc454e4438f44e", 
    "0x53d284357ec70cE289D6D64134DfAc8E511c8a3D"
]
address_tags = get_address_tags_from_etherscan(addresses)
for address, tags in address_tags.items():
    print(f"Address: {address}, Tags: {tags}")

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import requests
from bs4 import BeautifulSoup

def get_address_tag_from_etherscan(address):
    """
    Get the tag of a single Ethereum address from Etherscan.
    """
    url = f"https://etherscan.io/address/{address}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        tag_element = soup.find("span", {"class": "u-label u-label--xs u-label--info rounded"})

        if tag_element:
            return tag_element.text.strip()
        else:
            return "No tag found"
    else:
        return f"Failed to retrieve page, status code: {response.status_code}"

def get_multiple_address_tags(addresses):
    """
    Get tags for multiple Ethereum addresses from Etherscan.
    """
    address_tags = {}
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        results = executor.map(get_address_tag_from_etherscan, addresses)
        
    for address, tag in zip(addresses, results):
        address_tags[address] = tag
    
    return address_tags

# Example usage
addresses = [
    "0x742d35Cc6634C0532925a3b844Bc454e4438f44e",
    "0x53d284357ec70cE289D6D64134DfAc8E511c8a3D",
    "0x5a52E96BACdAbDcf4b16a7F8cE7C6c31E3Cc4aF6"
]

address_tags = get_multiple_address_tags(addresses)
for address, tag in address_tags.items():
    print(f"Address: {address}, Tag: {tag}")

In [ ]:
from dune_tools import get_address_tag_from_etherscan

result=get_address_tag_from_etherscan.invoke({"address":"0xF838ac8921e8d4Efdd20e605C2855f20691ba6a3"})
print(result)
result=get_address_tag_from_etherscan.invoke({"address":"0xA0c68C638235ee32657e8f720a23ceC1bFc77C77"})
print(result)
result=get_address_tag_from_etherscan.invoke({"address":"0x59a19d8c652fa0284f44113d0ff9aba70bd46fb4"})
print(result)

In [ ]:
from dune_tools import get_token_balances_of_address,extract_main_token_of_balances

# balances=get_token_balances_of_address.invoke({"address":'0xdfd5293d8e347dfe59e90efd55b2956a1343963d'})
# print(balances)
# result=extract_main_token_of_balances(balances)
# print(result)
from dune_tools import extract_token_from_balances
print(extract_token_from_balances(data=balances))

In [ ]:
from dune_tools import get_erc20_transfer_of

result=get_erc20_transfer_of.invoke({"address":'0xdfd5293d8e347dfe59e90efd55b2956a1343963d','contract_address':'0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'})
print(result)

In [ ]:
from dune_tools import get_eth_movements_of

result=get_eth_movements_of.invoke({"address":'0xdfd5293d8e347dfe59e90efd55b2956a1343963d'})
print(result)

In [ ]:
import pprint
from ethereum_address_analysis_agent import ethereum_address_analysis_agent_executor

# ethereum_address_analysis_agent_executor.invoke(
#     {
#         "input": "请帮我分析一下地址0xdfd5293d8e347dfe59e90efd55b2956a1343963d",
#         "chat_history": [],
#     },
#     config={
#         "configurable": {"llm": "openai_gpt_4o"},
#     },
# )

chunks = []
async for chunk in ethereum_address_analysis_agent_executor.astream_events(
    {
        "input": "请帮我分析一下地址0xdfd5293d8e347dfe59e90efd55b2956a1343963d",
        "chat_history": [],
    },
    version="v1",
    config={
        "configurable": {"llm": "openai_gpt_4o"},
    },
):
    chunks.append(chunk)
    print("------")
    pprint.pprint(chunk, depth=1)

In [ ]:
from dune_tools import get_balances_of_address

result=get_balances_of_address.invoke({"address":"0xdfd5293d8e347dfe59e90efd55b2956a1343963d"})

In [ ]:
pip install matplotlib

In [ ]:
from dune_tools import get_token_balance_daily_of_address

result=get_token_balance_daily_of_address.invoke({"address":"0xdfd5293d8e347dfe59e90efd55b2956a1343963d","token_address":"0x4575f41308ec1483f3d399aa9a2826d74da13deb"})
print(result)

In [ ]:
from tools_basic import tools
import json

t_arr=[{
            "name": tool.name,
            "description": tool.description,
            "parameters": {
                "type": "string",  # 假设工具接收的参数类型是字符串
                "format": f"{tool.name}({{{tool.name}_input}})"  # 定义参数格式
            }
        } for tool in tools]

print(json.dumps(t_arr))

In [ ]:
from dune_tools import get_address_labels

result=get_address_labels.invoke({"addresses":["0xdfd5293d8e347dfe59e90efd55b2956a1343963d"]})
print(result)

In [ ]:
from dune_tools import get_addres_how_much_funds_transfered

result = get_addres_how_much_funds_transfered.invoke(
    {
        "address": "0xdfd5293d8e347dfe59e90efd55b2956a1343963d",
        "contract_address": "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48",
        "token_sybmol": "USDC",
        "sta_time": "2024-07-29 00:00:00",
        "end_time": "2024-08-06 23:59:59",
    }
)
print(result)

In [6]:
s='\n\n感谢您的耐心。让我为您详细解释一下EigenLayer和Eigenpie的概念及其在DeFi生态系统中的重要作用。\n\nEigenLayer是一个创新的以太坊扩展协议，它引入了"重质押"（restaking）的概念。这个协议允许以太坊质押者将他们的质押资产重新质押到其他服务中，从而增加他们的收益潜力。EigenLayer的主要目标是提高以太坊网络的安全性和可扩展性，同时为质押者创造更多的价值。\n\nEigenLayer的关键特点包括：\n\n1. 重质押机制：允许用户在不解除原始质押的情况下，将ETH或流动性质押代币（LSTs）再次质押。\n\n2. 增强网络安全：通过增加质押资产的经济价值，提高攻击网络的成本。\n\n3. 支持创新服务：为新的去中心化服务（如数据可用性、预言机等）提供安全保障。\n\n4. 提高资本效率：让质押资产可以同时为多个服务提供安全保障。\n\nEigenpie是建立在EigenLayer之上的一个流动性再质押协议。它的主要功能和特点包括：\n\n1. 再质押服务：允许用户将他们的ETH或LSTs再次质押，以获得额外收益。\n\n2. 流动性优化：通过智能合约和算法，优化用户的质押策略，以最大化收益。\n\n3. 风险管理：提供多样化的质押选项，帮助用户平衡风险和收益。\n\n4. 治理token：EGP（Eigenpie的原生代币）允许持有者参与协议治理和决策。\n\n在DeFi生态系统中，EigenLayer和Eigenpie扮演着重要角色：\n\n1. 提高资本效率：通过重质押机制，使得相同数量的资产可以为多个协议提供安全保障，提高整个生态系统的资本效率。\n\n2. 促进创新：为新兴的去中心化服务提供安全基础设施，促进更多创新项目的发展。\n\n3. 增强网络安全：通过增加攻击成本，提高以太坊网络的整体安全性。\n\n4. 为用户创造价值：为质押者提供更多的收益机会，吸引更多用户参与DeFi生态系统。\n\n5. 推动DeFi发展：通过提供新的金融工具和服务，推动整个DeFi行业的发展和成熟。\n\n现在，让我们回到新闻的核心内容。Eigenpie将在PancakeSwap上进行IFO，这是一个重要的里程碑。IFO（Initial Farm Offering）是PancakeSwap独特的代币发行机制，允许新项目通过农场挖矿的方式向社区发行代币。\n\n这次IFO的几个关键点值得我们注意：\n\n1. 首次在Arbitrum网络上进行：这标志着PancakeSwap正在扩展其业务范围，不再局限于币安智能链。\n\n2. 时间安排：IFO将于2024年9月24日10:15（UTC）开始，持续24小时。\n\n3. 参与方式：包括面向所有CAKE持有者的基础IFO，以及为veCAKE持有者提供的优惠公开IFO。\n\n4. Eigenpie的规模：新闻中提到Eigenpie的总锁仓价值（TVL）超过10亿美元，这表明该项目已经获得了相当大的市场认可。\n\n5. EGP代币：作为Eigenpie的原生治理代币，EGP将在此次IFO中发行。持有者可以参与协议治理、获得潜在奖励，并通过锁定投票（vlEGP）增强他们在生态系统中的影响力。\n\n这次IFO对于加密货币市场有几个重要影响：\n\n1. 跨链发展：PancakeSwap在Arbitrum网络上推出IFO，表明DeFi项目正在积极探索跨链合作，这可能会推动不同区块链生态系统之间的互操作性。\n\n2. 再质押概念的普及：Eigenpie的IFO将进一步推广重质押的概念，可能会吸引更多投资者关注这种新型的DeFi策略。\n\n3. 流动性提升：成功的IFO可能会为Arbitrum网络带来更多流动性，促进该网络上DeFi生态系统的发展。\n\n4. 投资机会：对于CAKE和veCAKE持有者来说，这是一个参与新项目早期阶段的机会。\n\n5. 市场情绪：大规模IFO的成功举办可能会提振整个加密货币市场的信心，特别是在DeFi领域。\n\n展望未来，我们可以预期：\n\n1. 更多创新型DeFi项目可能会选择通过类似的IFO机制进行代币发行。\n\n2. 重质押和流动性优化策略可能会成为DeFi 2.0的重要组成部分。\n\n3. 跨链合作和互操作性将继续成为行业发展的重点。\n\n4. 投资者可能需要更加关注项目的技术创新和实际应用，而不仅仅是短期收益。\n\n总的来说，Eigenpie在PancakeSwap上的IFO代表了DeFi领域的一个重要发展，体现了市场对创新金融工具的需求，以及主要平台不断拓展业务范围的趋势。对于投资者来说，这既是一个机遇，也需要谨慎评估相关风险。\n\n感谢您收看Mlion的新闻分析，我是Simba。我们将继续为您带来最新、最深入的加密货币市场分析。再见！'
print(s)



感谢您的耐心。让我为您详细解释一下EigenLayer和Eigenpie的概念及其在DeFi生态系统中的重要作用。

EigenLayer是一个创新的以太坊扩展协议，它引入了"重质押"（restaking）的概念。这个协议允许以太坊质押者将他们的质押资产重新质押到其他服务中，从而增加他们的收益潜力。EigenLayer的主要目标是提高以太坊网络的安全性和可扩展性，同时为质押者创造更多的价值。

EigenLayer的关键特点包括：

1. 重质押机制：允许用户在不解除原始质押的情况下，将ETH或流动性质押代币（LSTs）再次质押。

2. 增强网络安全：通过增加质押资产的经济价值，提高攻击网络的成本。

3. 支持创新服务：为新的去中心化服务（如数据可用性、预言机等）提供安全保障。

4. 提高资本效率：让质押资产可以同时为多个服务提供安全保障。

Eigenpie是建立在EigenLayer之上的一个流动性再质押协议。它的主要功能和特点包括：

1. 再质押服务：允许用户将他们的ETH或LSTs再次质押，以获得额外收益。

2. 流动性优化：通过智能合约和算法，优化用户的质押策略，以最大化收益。

3. 风险管理：提供多样化的质押选项，帮助用户平衡风险和收益。

4. 治理token：EGP（Eigenpie的原生代币）允许持有者参与协议治理和决策。

在DeFi生态系统中，EigenLayer和Eigenpie扮演着重要角色：

1. 提高资本效率：通过重质押机制，使得相同数量的资产可以为多个协议提供安全保障，提高整个生态系统的资本效率。

2. 促进创新：为新兴的去中心化服务提供安全基础设施，促进更多创新项目的发展。

3. 增强网络安全：通过增加攻击成本，提高以太坊网络的整体安全性。

4. 为用户创造价值：为质押者提供更多的收益机会，吸引更多用户参与DeFi生态系统。

5. 推动DeFi发展：通过提供新的金融工具和服务，推动整个DeFi行业的发展和成熟。

现在，让我们回到新闻的核心内容。Eigenpie将在PancakeSwap上进行IFO，这是一个重要的里程碑。IFO（Initial Farm Offering）是PancakeSwap独特的代币发行机制，允许新项目通过农场挖矿的方式向社区发行代币。

这次IFO的几个关键点值得我们注意：

1.